<a href="https://colab.research.google.com/github/LipsaBiswas/GradioSummaryComparer/blob/main/GradioTestApp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# !pip install -q gradio

In [12]:
# !pip install -q transformers

In [13]:
# !pip install -q sentencepiece

In [14]:
# !pip install -q bert-extractive-summarizer 

In [15]:
import gradio as gr
import transformers
import sentencepiece
from summarizer import Summarizer

In [16]:
bert_model=Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [32]:
# model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-pubmed")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-pubmed")

In [23]:
# from transformers import pipeline

In [24]:
# summarizer = pipeline("summarization")

In [36]:
def SummarizeText(inp):  
   
    summary_generated = bert_model(inp)
#     print(summary_generated)
    print(inp)
    inputs = tokenizer.encode("summarize: " + inp, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(
        inputs, 
        max_length=128, 
        min_length=10, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True)
    summary_generated_pegasus_pubmed = tokenizer.decode(outputs[0])
    return summary_generated , summary_generated_pegasus_pubmed

In [38]:
iface = gr.Interface(
  fn=SummarizeText, 
  inputs=[gr.inputs.Textbox(label="Abstract (*Enter paragraphs)",lines=10, placeholder="Enter abstract text here...") ],
    
  outputs=[
           gr.outputs.Textbox(label=" Summary-1  (Bert)") ,
           gr.outputs.Textbox(label=" Summary-2 (google/pegasus-pubmed) ") ],title= "Auto summarization and Summary comparision(beta)"  )
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://28201.gradio.app
Interface loading below...


The outbreak of 2019 novel coronavirus (COVID-19) has caused serious threat to public health. Discovery of new anti-COVID-19 drugs is urgently needed. Fortunately, the crystal structure of COVID-19 3CL proteinase was recently resolved. The proteinase has been identified as a promising target for drug discovery in this crisis. Here, a dataset including 2030 natural compounds was screened and refined based on the machine learning and molecular docking. The performance of six machine learning (ML) methods of predicting active coronavirus inhibitors had achieved satisfactory accuracy, especially, the AUC (Area Under ROC Curve) scores with fivefold cross-validation of Logistic Regression (LR) reached up to 0.976. Comprehensive ML prediction and molecular docking results accounted for the compound Rutin, which was approved by NMPA (National Medical Products Administration), exhibited the best AUC and the most promising binding affinity compared to other compounds. Therefore, Rutin might be a

KeyboardInterrupt: ignored